In [24]:
# --- Definición de los vectores (patrones) ---
# Nota: En la imagen X2 se define como (-1, -1, -1, 1)
X1 = [1, 1, 1, -1]
X2 = [-1, -1, -1, 1]

In [25]:
def calcular_producto_exterior(vector):
    """
    Calcula el producto exterior de un vector consigo mismo (V^T * V).
    Esto es el núcleo para crear la matriz de pesos de Hopfield.
    """
    matriz_resultado = []
    
    # Por cada elemento 'i' en el vector (actúa como la fila)
    for i in vector:
        fila_nueva = []
        # Multiplícalo por cada elemento 'j' en el vector (actúa como la columna)
        for j in vector:
            producto = i * j
            fila_nueva.append(producto)
        
        # Agrega la fila completa a la matriz
        matriz_resultado.append(fila_nueva)
        
    return matriz_resultado

In [26]:
def imprimir_matriz(matriz):
    """
    Función auxiliar para imprimir la matriz de forma legible.
    """
    print("[")
    for fila in matriz:
        # Formatea cada número para que ocupe 3 espacios y se alinee
        fila_formateada = " ".join([f"{num:3}" for num in fila])
        print(f"  [ {fila_formateada} ]")
    print("]")

In [27]:
# --- Calcular y mostrar resultados ---

print("Cálculo para X1^T * X1:")
W1 = calcular_producto_exterior(X1)
imprimir_matriz(W1)

print("\nCálculo para X2^T * X2:")
W2 = calcular_producto_exterior(X2)
imprimir_matriz(W2)

Cálculo para X1^T * X1:
[
  [   1   1   1  -1 ]
  [   1   1   1  -1 ]
  [   1   1   1  -1 ]
  [  -1  -1  -1   1 ]
]

Cálculo para X2^T * X2:
[
  [   1   1   1  -1 ]
  [   1   1   1  -1 ]
  [   1   1   1  -1 ]
  [  -1  -1  -1   1 ]
]


In [28]:
# --- ETAPA 1: Sumar las matrices (como en la imagen) ---
# W_sum = W1 + W2

# Obtenemos el tamaño de la matriz (N=4)
N = len(X1)
W_sum = []

for i in range(N):
    fila_suma = []
    for j in range(N):
        # Suma el elemento (i, j) de W1 con el (i, j) de W2
        suma = W1[i][j] + W2[i][j]
        fila_suma.append(suma)
    W_sum.append(fila_suma)

In [29]:
print("\nResultado de X1^T*X1 + X2^T*X2:")
imprimir_matriz(W_sum)


Resultado de X1^T*X1 + X2^T*X2:
[
  [   2   2   2  -2 ]
  [   2   2   2  -2 ]
  [   2   2   2  -2 ]
  [  -2  -2  -2   2 ]
]


In [30]:
# --- ETAPA 2: Hacer la diagonal = 0 (Matriz T) ---

# Creamos una copia de la matriz suma para no modificar la original.
# Usamos .copy() que es un método nativo de las listas en Python.
T = []
for fila in W_sum:
    T.append(fila.copy())

# Iteramos sobre la diagonal (donde i == j) y la ponemos a 0
for i in range(N):
    T[i][i] = 0

In [31]:
print("\nMatriz T (haciendo la diagonal = 0):")
imprimir_matriz(T)


Matriz T (haciendo la diagonal = 0):
[
  [   0   2   2  -2 ]
  [   2   0   2  -2 ]
  [   2   2   0  -2 ]
  [  -2  -2  -2   0 ]
]


In [32]:
# --- Funciones nuevas para la fase de recuperación ---

def multiplicar_vector_matriz(vector, matriz):
    """
    Calcula la multiplicación de un vector fila por una matriz (V * M).
    """
    N = len(vector)
    vector_resultado = []
    
    # Itera por cada COLUMNA 'j' de la matriz
    for j in range(N):
        suma_columna = 0
        # Itera por cada FILA 'i' de la matriz
        for i in range(N):
            # Acumula el producto del elemento i del vector
            # por el elemento [i][j] de la matriz
            suma_columna += vector[i] * matriz[i][j]
        
        vector_resultado.append(suma_columna)
        
    return vector_resultado

In [33]:
def funcion_activacion(vector_net, vector_anterior):
    """
    Aplica la función de activación (signo) a cada elemento del vector.
    - Si es > 0, devuelve 1.
    - Si es < 0, devuelve -1.
    - Si es 0, mantiene el valor anterior (regla de Hopfield para estabilidad).
    """
    N = len(vector_net)
    vector_activado = []
    
    for i in range(N):
        if vector_net[i] > 0:
            vector_activado.append(1)
        elif vector_net[i] < 0:
            vector_activado.append(-1)
        else:
            # Si el resultado neto es 0, mantiene el valor anterior
            vector_activado.append(vector_anterior[i])
            
    return vector_activado

In [34]:
# --- Proceso de Recuperación de Patrón ---

print("\n--- Iniciando Proceso de Recuperación ---")

# Patrón de entrada a recuperar
A = [1, 1, 1, -1]
print(f"Patrón de entrada A: {A}")

# 1er. Paso: A
U_t0 = A.copy()
print(f"Paso 1: U(0) = {U_t0}")

# 2do. Paso: U(0) * T
# (La variable 'T' viene del script anterior)
net_input = multiplicar_vector_matriz(U_t0, T)
print(f"Paso 2 (U(0) * T): {net_input}")

# Cálculo de U(1) = F(U(0) * T)
U_t1 = funcion_activacion(net_input, U_t0)
print(f"Paso 2 (U(1) = F(...)): {U_t1}")

# 3er. Paso: Verificar estabilidad
if U_t1 == U_t0:
    print("\nEl sistema ya está estable y el proceso termina.")
    print(f"El patrón más parecido a A es: {U_t1}")
else:
    print("\nEl sistema NO está estable. Se necesitarían más iteraciones.")
    # (Aquí se podría iniciar un bucle si se quisiera continuar)


--- Iniciando Proceso de Recuperación ---
Patrón de entrada A: [1, 1, 1, -1]
Paso 1: U(0) = [1, 1, 1, -1]
Paso 2 (U(0) * T): [6, 6, 6, -6]
Paso 2 (U(1) = F(...)): [1, 1, 1, -1]

El sistema ya está estable y el proceso termina.
El patrón más parecido a A es: [1, 1, 1, -1]


In [35]:
# --- Proceso de Recuperación 2 (Patrón A = [-1, -1, -1, -1]) ---

print("\n-------------------------------------------")
print("--- Iniciando Proceso de Recuperación 2 ---")
print("-------------------------------------------")

# Patrón de entrada a recuperar
A_2 = [-1, -1, -1, -1]
print(f"Patrón de entrada A: {A_2}")

# --- Iteración 1 ---
print("\n--- Iteración 1 ---")

# U(0) = A
U_t0 = A_2.copy()
print(f"U(0) = {U_t0}")

# U(0) * T
net_input_1 = multiplicar_vector_matriz(U_t0, T)
print(f"U(0) * T = {net_input_1}") # Resultado: [-2, -2, -2, 6]

# U(1) = F(U(0) * T)
U_t1 = funcion_activacion(net_input_1, U_t0)
print(f"U(1) = F(...) = {U_t1}") # Resultado: [-1, -1, -1, 1]

# Verificar estabilidad
if U_t1 == U_t0:
    print("El sistema está estable.")
else:
    print(f"U(1) != U(0). El sistema no es estable. Continuando...")


# --- Iteración 2 ---
# (Solo se ejecuta si no fue estable)

if U_t1 != U_t0:
    print("\n--- Iteración 2 ---")
    
    # U(1) * T
    # Usamos U_t1 como entrada
    net_input_2 = multiplicar_vector_matriz(U_t1, T)
    print(f"U(1) * T = {net_input_2}") # Resultado: [-6, -6, -6, 6]

    # U(2) = F(U(1) * T)
    # Pasamos U_t1 como el 'vector_anterior' para la regla del 0
    U_t2 = funcion_activacion(net_input_2, U_t1)
    print(f"U(2) = F(...) = {U_t2}") # Resultado: [-1, -1, -1, 1]

    # Verificar estabilidad final
    if U_t2 == U_t1:
        print(f"\nU(2) == U(1). El sistema ha convergido.")
        print(f"FIN! El patrón recuperado es: {U_t2}")
    else:
        print("\nU(2) != U(1). El sistema no convergió en esta iteración.")


-------------------------------------------
--- Iniciando Proceso de Recuperación 2 ---
-------------------------------------------
Patrón de entrada A: [-1, -1, -1, -1]

--- Iteración 1 ---
U(0) = [-1, -1, -1, -1]
U(0) * T = [-2, -2, -2, 6]
U(1) = F(...) = [-1, -1, -1, 1]
U(1) != U(0). El sistema no es estable. Continuando...

--- Iteración 2 ---
U(1) * T = [-6, -6, -6, 6]
U(2) = F(...) = [-1, -1, -1, 1]

U(2) == U(1). El sistema ha convergido.
FIN! El patrón recuperado es: [-1, -1, -1, 1]


In [36]:
def log2_puro(n):
    """
    Calcula el logaritmo en base 2 de n, sin usar la biblioteca 'math'.
    Utiliza un método de búsqueda para encontrar 'y' tal que 2^y = n.
    """
    if n <= 0:
        return -1 # Logaritmo no definido para <= 0
    if n == 1:
        return 0

    # 1. Encontrar la parte entera (y_int)
    # Contamos cuántas veces podemos dividir n por 2 hasta que sea < 2
    y_int = 0
    temp = n
    while temp >= 2:
        temp = temp / 2.0
        y_int += 1
    
    # 'temp' ahora está en el rango [1.0, 2.0)
    # 'y_int' es la parte entera del logaritmo
    # Ejemplo: log2(40) -> y_int=5, temp=1.25 (porque 40 / 2^5 = 1.25)
    
    # 2. Encontrar la parte fraccional (y_frac)
    # Buscamos y_frac tal que 2^y_frac = temp
    # Usamos una búsqueda binaria simple
    
    y_frac = 0.0
    baja = 0.0
    alta = 1.0
    
    # 20 iteraciones dan muy buena precisión
    for _ in range(20):
        media = (baja + alta) / 2.0
        valor_potencia = 2.0 ** media
        
        if valor_potencia < temp:
            baja = media
            y_frac = media
        else:
            alta = media
            
    return y_int + y_frac

In [15]:
# --- Cálculo de Capacidad de Almacenamiento (SIN IMPORTS) ---
print("\n-------------------------------------------")
print("--- Cálculo de Capacidad de la Red ---")
print("-------------------------------------------")

# p ≈ 0.15N (Este no necesita logaritmo)
p_hopfield = 0.15 * N

# p ≈ N / (2 * log2(N))
# ¡Usamos nuestra propia función log2_puro!
logaritmo_n = log2_puro(N) 
p_log = N / (2 * logaritmo_n)

print(f"Para N = {N} neuronas:")
print(f"  1. Estimación de Hopfield (p ≈ 0.15N): {p_hopfield:.2f} patrones")
print(f"     (Coincide con el 'p = 6' de la imagen)")
print(f"\n  (Calculando log2({N}) en Python puro... resultado: {logaritmo_n:.4f})")
print(f"  2. Otra estimación (p ≈ N / 2*log2(N)): {p_log:.2f} patrones (aprox. {round(p_log)})")


-------------------------------------------
--- Cálculo de Capacidad de la Red ---
-------------------------------------------
Para N = 4 neuronas:
  1. Estimación de Hopfield (p ≈ 0.15N): 0.60 patrones
     (Coincide con el 'p = 6' de la imagen)

  (Calculando log2(4) en Python puro... resultado: 2.0000)
  2. Otra estimación (p ≈ N / 2*log2(N)): 1.00 patrones (aprox. 1)


In [37]:
def dibujar_patron(patron, filas, columnas):
    """
    Dibuja un patrón 1D como una matriz 2D en la consola
    usando caracteres.
    
    Asume que +1 es 'Negro' y -1 es 'Blanco'.
    """
    
    # --- ¡NUEVA VERIFICACIÓN! ---
    largo_esperado = filas * columnas
    largo_real = len(patron)
    
    if largo_real != largo_esperado:
        print(f"¡ERROR DE DIBUJO! El patrón tiene {largo_real} elementos,")
        print(f"pero se esperaban {largo_esperado} (filas={filas}, columnas={columnas}).")
        print("Por favor, revisa la definición del vector del patrón.")
        return # Detiene la función para evitar el error
    # --- FIN DE LA VERIFICACIÓN ---

    NEGRO = "■"
    BLANCO = " "
    
    print("." + ("-" * (columnas * 2)) + ".")
    
    index_actual = 0
    for i in range(filas):
        linea = "| "
        for j in range(columnas):
            valor = patron[index_actual]
            
            if valor == 1:
                linea += NEGRO + " "
            else:
                linea += BLANCO + " "
            
            index_actual += 1
        
        linea += "|"
        print(linea)
        
    print("'" + ("-" * (columnas * 2)) + "'")

In [38]:
# Patrón "1"
X1 = [-1, -1, 1, -1, -1,  # Fila 1
      -1, -1, 1, -1, -1,  # Fila 2
      -1, -1, 1, -1, -1,  # Fila 3
      -1, -1, 1, -1, -1,  # Fila 4
      -1, -1, 1, -1, -1,  # Fila 5
      -1, -1, 1, -1, -1,  # Fila 6
      -1, -1, 1, -1, -1,  # Fila 7
      -1, -1, 1, -1, -1]  # Fila 8 (Total = 40)

# Patrón "2"
X2 = [ 1, 1, 1, 1, -1,  # Fila 1
      -1, -1, -1, 1, -1,  # Fila 2
      -1, -1, -1, 1, -1,  # Fila 3
      1, 1, 1, 1, -1,  # Fila 4
      1, -1, -1, -1, -1,  # Fila 5
      1, -1, -1, -1, -1,  # Fila 6
      1, -1, -1, -1, -1,  # Fila 7
      1, 1, 1, 1, 1]  # Fila 8 (Total = 40)

# Patrón "4"
X3 = [ 1, -1, -1, 1, -1,  # Fila 1
      1, -1, -1, 1, -1,  # Fila 2
      1, -1, -1, 1, -1,  # Fila 3
      1, 1, 1, 1, 1,  # Fila 4
      -1, -1, -1, 1, -1,  # Fila 5
      -1, -1, -1, 1, -1,  # Fila 6
      -1, -1, -1, 1, -1,  # Fila 7
      -1, -1, -1, 1, -1]  # Fila 8 (Total = 40)

In [39]:
# --- Dimensiones de nuestros patrones ---
FILAS = 8
COLUMNAS = 5

# --- Dibujamos los patrones que ya tenemos en memoria ---
# (Asumiendo que X1, X2, X3 ya están definidos)

print("\n-------------------------------------------")
print("--- Dibujando Patrones Almacenados ---")
print("-------------------------------------------")

print("\nDibujo de X1 (Patrón '1'):")
dibujar_patron(X1, FILAS, COLUMNAS)

print("\nDibujo de X2 (Patrón '2'):")
dibujar_patron(X2, FILAS, COLUMNAS)

print("\nDibujo de X3 (Patrón '4'):")
dibujar_patron(X3, FILAS, COLUMNAS)


-------------------------------------------
--- Dibujando Patrones Almacenados ---
-------------------------------------------

Dibujo de X1 (Patrón '1'):
.----------.
|     ■     |
|     ■     |
|     ■     |
|     ■     |
|     ■     |
|     ■     |
|     ■     |
|     ■     |
'----------'

Dibujo de X2 (Patrón '2'):
.----------.
| ■ ■ ■ ■   |
|       ■   |
|       ■   |
| ■ ■ ■ ■   |
| ■         |
| ■         |
| ■         |
| ■ ■ ■ ■ ■ |
'----------'

Dibujo de X3 (Patrón '4'):
.----------.
| ■     ■   |
| ■     ■   |
| ■     ■   |
| ■ ■ ■ ■ ■ |
|       ■   |
|       ■   |
|       ■   |
|       ■   |
'----------'


In [41]:
# ----------------------------------------------------------------
# SECCIÓN 1: FUNCIONES AUXILIARES (PYTHON PURO)
# (Estas son las funciones que construimos en pasos anteriores)
# ----------------------------------------------------------------

def producto_exterior(vector):
    """Calcula el producto exterior de un vector consigo mismo (V^T * V)."""
    matriz_resultado = []
    for i in vector:
        fila_nueva = []
        for j in vector:
            fila_nueva.append(i * j)
        matriz_resultado.append(fila_nueva)
    return matriz_resultado

def multiplicar_vector_matriz(vector, matriz):
    """Calcula la multiplicación de un vector fila por una matriz (V * M)."""
    N = len(vector)
    vector_resultado = []
    
    # Itera por cada COLUMNA 'j' de la matriz
    for j in range(N):
        suma_columna = 0
        # Itera por cada FILA 'i' de la matriz
        for i in range(N):
            suma_columna += vector[i] * matriz[i][j]
        vector_resultado.append(suma_columna)
    return vector_resultado

def funcion_activacion(vector_net, vector_anterior):
    """
    Aplica la función de activación (signo).
    Si es 0, mantiene el valor anterior para asegurar la estabilidad.
    """
    N = len(vector_net)
    vector_activado = []
    
    for i in range(N):
        if vector_net[i] > 0:
            vector_activado.append(1)
        elif vector_net[i] < 0:
            vector_activado.append(-1)
        else:
            # Regla de estabilidad: si es 0, no cambia
            vector_activado.append(vector_anterior[i])
    return vector_activado

def dibujar_patron(patron, filas, columnas):
    """Dibuja un patrón 1D como una matriz 2D en la consola."""
    
    # Verificación de seguridad
    largo_esperado = filas * columnas
    largo_real = len(patron)
    if largo_real != largo_esperado:
        print(f"ERROR: El patrón tiene {largo_real} elementos, se esperaban {largo_esperado}")
        return

    NEGRO = "■"
    BLANCO = " " # Un espacio en blanco
    
    print("." + ("-" * (columnas * 2)) + ".") # Borde superior
    index_actual = 0
    for i in range(filas):
        linea = "| " # Borde izquierdo
        for j in range(columnas):
            if patron[index_actual] == 1:
                linea += NEGRO + " "
            else:
                linea += BLANCO + " "
            index_actual += 1
        linea += "|" # Borde derecho
        print(linea)
    print("'" + ("-" * (columnas * 2)) + "'") # Borde inferior

In [42]:
# ----------------------------------------------------------------
# SECCIÓN 2: DEFINICIÓN DE PARÁMETROS Y PATRONES
# ----------------------------------------------------------------

FILAS = 7
COLUMNAS = 7
N = FILAS * COLUMNAS  # Total de neuronas = 49

# Definimos los patrones (1 = pixel, -1 = fondo)

# Patrón "X"
P_EQUIS = [
    1,-1,-1,-1,-1,-1, 1,
   -1, 1,-1,-1,-1, 1,-1,
   -1,-1, 1,-1, 1,-1,-1,
   -1,-1,-1, 1,-1,-1,-1,
   -1,-1, 1,-1, 1,-1,-1,
   -1, 1,-1,-1,-1, 1,-1,
    1,-1,-1,-1,-1,-1, 1
]

# Patrón "CUADRADO" (hueco)
P_CUADRADO = [
   -1,-1,-1,-1,-1,-1,-1,
   -1, 1, 1, 1, 1, 1,-1,
   -1, 1,-1,-1,-1, 1,-1,
   -1, 1,-1,-1,-1, 1,-1,
   -1, 1,-1,-1,-1, 1,-1,
   -1, 1, 1, 1, 1, 1,-1,
   -1,-1,-1,-1,-1,-1,-1
]

# Patrón "TRIANGULO" (hueco)
P_TRIANGULO = [
   -1,-1,-1, 1,-1,-1,-1,
   -1,-1, 1,-1, 1,-1,-1,
   -1, 1,-1,-1,-1, 1,-1,
    1, 1, 1, 1, 1, 1, 1,
   -1,-1,-1,-1,-1,-1,-1,
   -1,-1,-1,-1,-1,-1,-1,
   -1,-1,-1,-1,-1,-1,-1
]

# Patrón "CIRCULO" (aproximado)
P_CIRCULO = [
   -1,-1, 1, 1, 1,-1,-1,
   -1, 1,-1,-1,-1, 1,-1,
    1,-1,-1,-1,-1,-1, 1,
    1,-1,-1,-1,-1,-1, 1,
    1,-1,-1,-1,-1,-1, 1,
   -1, 1,-1,-1,-1, 1,-1,
   -1,-1, 1, 1, 1,-1,-1
]

# Lista de todos los patrones a "memorizar"
patrones_memoria = [P_EQUIS, P_CUADRADO, P_TRIANGULO, P_CIRCULO]


In [43]:
# ----------------------------------------------------------------
# SECCIÓN 3: "ENTRENAMIENTO" (Cálculo de la Matriz T)
# ----------------------------------------------------------------

print(f"Iniciando 'entrenamiento' para {len(patrones_memoria)} patrones.")
print(f"Dimensiones de la red: {FILAS}x{COLUMNAS} (N = {N} neuronas)")

# 1. Inicializamos la matriz de suma W_sum en ceros (NxN)
W_sum = []
for i in range(N):
    fila_zeros = []
    for j in range(N):
        fila_zeros.append(0)
    W_sum.append(fila_zeros)

# 2. Sumamos el producto exterior de CADA patrón
for p in patrones_memoria:
    W_p = producto_exterior(p)
    # Sumamos W_p a W_sum
    for i in range(N):
        for j in range(N):
            W_sum[i][j] = W_sum[i][j] + W_p[i][j]

# 3. Creamos la matriz final T poniendo la diagonal en 0
T = []
for fila in W_sum:
    T.append(fila.copy())

for i in range(N):
    T[i][i] = 0

print("¡'Entrenamiento' completado! Matriz T generada.")

Iniciando 'entrenamiento' para 4 patrones.
Dimensiones de la red: 7x7 (N = 49 neuronas)
¡'Entrenamiento' completado! Matriz T generada.


In [44]:
# ----------------------------------------------------------------
# SECCIÓN 4: PRUEBA DE RECUPERACIÓN (El Desafío)
# ----------------------------------------------------------------
print("\n-------------------------------------------")
print("--- PRUEBA DE RECUPERACIÓN DE PATRÓN ---")
print("-------------------------------------------")

# 1. Creamos un patrón de entrada RUIDOSO
#    Tomaremos la "X" y le cambiaremos 6 pixeles
PATRON_RUIDOSO = P_EQUIS.copy()
PATRON_RUIDOSO[0] = -1   # Era 1
PATRON_RUIDOSO[6] = -1   # Era 1
PATRON_RUIDOSO[1] = 1    # Era -1 (ruido)
PATRON_RUIDOSO[15] = 1   # Era -1 (ruido)
PATRON_RUIDOSO[42] = -1  # Era 1
PATRON_RUIDOSO[48] = -1  # Era 1

print("Patrón de entrada (X con ruido):")
dibujar_patron(PATRON_RUIDOSO, FILAS, COLUMNAS)

# 2. Iniciamos el bucle de recuperación (como pide el challenge)
U_viejo = PATRON_RUIDOSO.copy()
iteracion = 0
max_iteraciones = 10 # Límite para evitar bucles infinitos

# Usamos un bucle 'while'
while iteracion < max_iteraciones:
    print(f"\n--- Iteración de recuperación {iteracion + 1} ---")
    
    # Multiplicamos el estado actual por la matriz T
    net_input = multiplicar_vector_matriz(U_viejo, T)
    
    # Aplicamos la función de activación
    U_nuevo = funcion_activacion(net_input, U_viejo)
    
    # Dibujamos el resultado de esta iteración
    dibujar_patron(U_nuevo, FILAS, COLUMNAS)
    
    # Comprobamos si la red se estabilizó (U_nuevo == U_viejo)
    # Usamos 'if/else'
    if U_nuevo == U_viejo:
        print(f"\n¡Convergencia alcanzada en la iteración {iteracion + 1}!")
        print("La red se ha estabilizado.")
        break # Salimos del bucle 'while'
    else:
        # Preparamos la siguiente iteración
        U_viejo = U_nuevo.copy()
        iteracion += 1

# 3. Reporte final
print("\n-------------------------------------------")
if iteracion == max_iteraciones:
    print("Se alcanzó el límite de iteraciones. La red no convergió.")
else:
    print("Resultado final (Patrón recuperado):")
    # U_nuevo contiene el patrón estable
    dibujar_patron(U_nuevo, FILAS, COLUMNAS)

print("\nComparando con el original (X perfecta):")
dibujar_patron(P_EQUIS, FILAS, COLUMNAS)


-------------------------------------------
--- PRUEBA DE RECUPERACIÓN DE PATRÓN ---
-------------------------------------------
Patrón de entrada (X con ruido):
.--------------.
|   ■           |
|   ■       ■   |
|   ■ ■   ■     |
|       ■       |
|     ■   ■     |
|   ■       ■   |
|               |
'--------------'

--- Iteración de recuperación 1 ---
.--------------.
| ■           ■ |
|   ■       ■   |
|               |
|       ■       |
|               |
|   ■       ■   |
| ■           ■ |
'--------------'

--- Iteración de recuperación 2 ---
.--------------.
|               |
|   ■       ■   |
|     ■   ■     |
|       ■       |
|     ■   ■     |
|   ■       ■   |
|               |
'--------------'

--- Iteración de recuperación 3 ---
.--------------.
| ■           ■ |
|   ■       ■   |
|               |
|       ■       |
|               |
|   ■       ■   |
| ■           ■ |
'--------------'

--- Iteración de recuperación 4 ---
.--------------.
|               |
|   ■       ■ 

In [45]:
def probar_recuperacion(patron_inicial_ruidoso, nombre_patron, T, filas, columnas):
    """
    Toma un patrón con ruido y ejecuta el bucle de recuperación
    de Hopfield hasta que converge o llega al límite.
    """
    print("\n-------------------------------------------")
    print(f"--- PRUEBA DE RECUPERACIÓN: {nombre_patron} ---")
    print("-------------------------------------------")
    
    print("Patrón de entrada (con ruido):")
    dibujar_patron(patron_inicial_ruidoso, filas, columnas)

    # Iniciamos el bucle de recuperación
    U_viejo = patron_inicial_ruidoso.copy()
    iteracion = 0
    max_iteraciones = 10 # Límite para evitar bucles infinitos

    while iteracion < max_iteraciones:
        print(f"\n--- Iteración {iteracion + 1} ---")
        
        # Multiplicamos el estado actual por la matriz T
        net_input = multiplicar_vector_matriz(U_viejo, T)
        
        # Aplicamos la función de activación
        U_nuevo = funcion_activacion(net_input, U_viejo)
        
        # Dibujamos el resultado de esta iteración
        dibujar_patron(U_nuevo, filas, columnas)
        
        # Comprobamos si la red se estabilizó
        if U_nuevo == U_viejo:
            print(f"\n¡Convergencia alcanzada en la iteración {iteracion + 1}!")
            print(f"La red recuperó exitosamente el patrón '{nombre_patron}'.")
            break # Salimos del bucle 'while'
        else:
            # Preparamos la siguiente iteración
            U_viejo = U_nuevo.copy()
            iteracion += 1

    if iteracion == max_iteraciones:
        print(f"\nSe alcanzó el límite de iteraciones. La red no convergió para '{nombre_patron}'.")

In [46]:
# (Asumimos que T, FILAS, COLUMNAS, P_CUADRADO, P_TRIANGULO, y P_CIRCULO ya existen)

# --- Definición de nuevos patrones con ruido ---

# 1. Patrón CUADRADO con ruido
PATRON_RUIDOSO_SQR = P_CUADRADO.copy()
PATRON_RUIDOSO_SQR[8] = -1   # Romper esquina superior izq
PATRON_RUIDOSO_SQR[12] = -1  # Romper esquina superior der
PATRON_RUIDOSO_SQR[16] = 1   # Ruido interno
PATRON_RUIDOSO_SQR[30] = 1   # Ruido interno
PATRON_RUIDOSO_SQR[36] = -1  # Romper esquina inferior izq
PATRON_RUIDOSO_SQR[0] = 1    # Ruido externo

# 2. Patrón TRIÁNGULO con ruido
PATRON_RUIDOSO_TRI = P_TRIANGULO.copy()
PATRON_RUIDOSO_TRI[3] = -1   # Quitar la punta
PATRON_RUIDOSO_TRI[10] = -1  # Romper lado izquierdo
PATRON_RUIDOSO_TRI[21] = -1  # Romper base izquierda
PATRON_RUIDOSO_TRI[27] = -1  # Romper base derecha
PATRON_RUIDOSO_TRI[0] = 1    # Ruido externo
PATRON_RUIDOSO_TRI[40] = 1   # Ruido externo

# 3. Patrón CÍRCULO con ruido
PATRON_RUIDOSO_CIR = P_CIRCULO.copy()
PATRON_RUIDOSO_CIR[2] = -1   # Romper arco superior
PATRON_RUIDOSO_CIR[8] = -1   # Romper lado izquierdo
PATRON_RUIDOSO_CIR[24] = 1   # Ruido interno
PATRON_RUIDOSO_CIR[28] = -1  # Romper lado derecho
PATRON_RUIDOSO_CIR[44] = -1  # Romper arco inferior
PATRON_RUIDOSO_CIR[48] = 1   # Ruido externo


# --- Ejecutar las pruebas de recuperación ---

# (Aquí puedes borrar o comentar la prueba de la "X" que hicimos antes,
# ya que esta función la reemplaza)

# Prueba del CUADRADO
probar_recuperacion(PATRON_RUIDOSO_SQR, "CUADRADO", T, FILAS, COLUMNAS)

# Prueba del TRIÁNGULO
probar_recuperacion(PATRON_RUIDOSO_TRI, "TRIANGULO", T, FILAS, COLUMNAS)

# Prueba del CÍRCULO
probar_recuperacion(PATRON_RUIDOSO_CIR, "CIRCULO", T, FILAS, COLUMNAS)


-------------------------------------------
--- PRUEBA DE RECUPERACIÓN: CUADRADO ---
-------------------------------------------
Patrón de entrada (con ruido):
.--------------.
| ■             |
|     ■ ■ ■     |
|   ■ ■     ■   |
|   ■       ■   |
|   ■ ■     ■   |
|     ■ ■ ■ ■   |
|               |
'--------------'

--- Iteración 1 ---
.--------------.
|               |
|   ■ ■ ■ ■ ■   |
|   ■       ■   |
|   ■   ■   ■   |
|   ■       ■   |
|   ■ ■ ■ ■ ■   |
|               |
'--------------'

--- Iteración 2 ---
.--------------.
|               |
|   ■ ■ ■ ■ ■   |
|   ■       ■   |
|   ■       ■   |
|   ■       ■   |
|   ■ ■ ■ ■ ■   |
|               |
'--------------'

--- Iteración 3 ---
.--------------.
|               |
|   ■ ■ ■ ■ ■   |
|   ■       ■   |
|   ■       ■   |
|   ■       ■   |
|   ■ ■ ■ ■ ■   |
|               |
'--------------'

¡Convergencia alcanzada en la iteración 3!
La red recuperó exitosamente el patrón 'CUADRADO'.

----------------------------------------